In [1]:
import pandas as pd
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import confusion_matrix
import torch
from datasets import Dataset, DatasetDict
import evaluate
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return clf_metrics.compute(predictions=predictions, references=labels)

In [2]:
df = pd.read_csv('entities.csv', sep=',')
df.rename(columns={'chef':'label', 'texte':'text'}, inplace=True)
df.label = df.label.astype('int')
df

,text,label
0,Breton Cyrille menuisier 25 Garçon française,0
1,Ferazzi Auguste vitrier 30 Garçon Piémontaise,1
2,Machol Pierre vitrier 24 Garçon Piémontaise,1
3,Desbois Alexandre prop re 48 Homme marié franç...,1
4,Vignat Zélie prop re sa fe 30 française,0
...,...,...
25075,Chameton-Dideron Marie chef 1869 idem Pailharès,1
25076,Ode Marie ouv chaus res chef Cara 1863 idem St...,1
25077,Berni Nello manoeuvre chef Baretto 1886 italie...,1
25078,Berni-Laureti Annunziata épouse 1887 idem idem,0


In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")


In [4]:
def preprocess_function(examples, batched = True):
    return tokenizer(examples["text"], truncation=True)

In [15]:
dataset = Dataset.from_pandas(df)

In [14]:
dataset["train"][0]

{'text': 'Rabier néant femme 1859 française', 'label': 0}

In [16]:
train_testvalid = dataset.train_test_split(0.2, seed=42)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(0.5)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

dataset = train_test_valid_dataset
tokenized_text = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/20064 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding = 'longest')

In [11]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

training_args = TrainingArguments(
    output_dir="distilbert",
    learning_rate=1e-6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16 = True)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
).to(device)

training_args = TrainingArguments(
    output_dir="distilbert",
    learning_rate=1e-6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16 = True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_text["train"],
    eval_dataset=tokenized_text["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/users/eleves-a/2020/meilame.tayebjee/miniconda3/lib/python3.12/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.415600,0.228706,0.917065,0.828947,0.791209,0.870466
2,0.239600,0.200801,0.926635,0.849673,0.806202,0.898100
3,0.221100,0.191919,0.931021,0.859464,0.811350,0.913644
4,0.204000,0.187840,0.933413,0.864558,0.814985,0.920553
5,0.194900,0.183977,0.934211,0.867043,0.812689,0.929188
6,0.199100,0.182843,0.933812,0.866559,0.810526,0.930915
7,0.192300,0.180792,0.933413,0.866932,0.804734,0.939551
8,0.185600,0.181291,0.935008,0.868654,0.814199,0.930915
9,0.191900,0.180251,0.934609,0.868800,0.809240,0.937824
10,0.183900,0.180036,0.934211,0.867682,0.809880,0.934370


TrainOutput(global_step=6270, training_loss=0.21743381019604452, metrics={'train_runtime': 369.9584, 'train_samples_per_second': 542.331, 'train_steps_per_second': 16.948, 'total_flos': 1194052852272768.0, 'train_loss': 0.21743381019604452, 'epoch': 10.0})

In [21]:
#Evaluate on test set 
predictions = trainer.predict(tokenized_text["test"])
print(np.mean(predictions.predictions.argmax(axis=1) != predictions.label_ids))



0.07177033492822966


In [23]:
confusion_matrix(predictions.label_ids, predictions.predictions.argmax(axis=1))

array([[1830,  147],
       [  33,  498]])

### Estimate variance

In [27]:
training_args = TrainingArguments(
    output_dir="distilbert",
    learning_rate=1e-6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="no",
    fp16 = True)

all_test_metrics = []

for seed in range(5):
    cv_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id).to(device)
    

    ##### Split the data #####
    dataset = Dataset.from_pandas(df)
    train_testvalid = dataset.train_test_split(0.2, seed=seed)
    # Split the 10% test + valid in half test, half valid
    test_valid = train_testvalid['test'].train_test_split(0.5)
    # gather everyone if you want to have a single DatasetDict
    train_test_valid_dataset = DatasetDict({
        'train': train_testvalid['train'],
        'test': test_valid['test'],
        'valid': test_valid['train']})

    dataset = train_test_valid_dataset
    tokenized_text = dataset.map(preprocess_function, batched=True)

    trainer = Trainer(
        model=cv_model,
        args=training_args,
        train_dataset=tokenized_text["train"],
        eval_dataset=tokenized_text["valid"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    #Evaluate on test set
    predictions = trainer.predict(tokenized_text["test"])
    all_test_metrics.append(predictions.metrics)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/20064 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

/users/eleves-a/2020/meilame.tayebjee/miniconda3/lib/python3.12/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


...

Map:   0%|          | 0/20064 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

/users/eleves-a/2020/meilame.tayebjee/miniconda3/lib/python3.12/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.385900,0.234158,0.907895,0.801034,0.761047,0.845455
2,0.239500,0.211402,0.920255,0.829352,0.781350,0.883636
3,0.213200,0.202619,0.924242,0.837050,0.792208,0.887273
4,0.198100,0.199678,0.926635,0.841924,0.798046,0.890909
5,0.192200,0.193613,0.927033,0.845570,0.788976,0.910909
6,0.196500,0.192572,0.929426,0.850380,0.794629,0.914545
7,0.188600,0.190747,0.929426,0.850633,0.793701,0.916364
8,0.188900,0.188840,0.927432,0.847059,0.787500,0.916364
9,0.181200,0.189435,0.929825,0.851351,0.794953,0.916364
10,0.186700,0.189231,0.929825,0.851351,0.794953,0.916364


In [37]:
res_table = pd.DataFrame(all_test_metrics)

full_res = pd.concat([res_table.mean(), res_table.std()], axis=1)
full_res.columns = ['mean', 'std']
full_res

,mean,std
test_loss,0.183553,0.013981
test_accuracy,0.931499,0.004755
test_f1,0.856752,0.008005
test_precision,0.793588,0.007768
test_recall,0.930851,0.008842
test_runtime,0.911480,0.004860
test_samples_per_second,2751.633200,14.688787
test_steps_per_second,172.251200,0.919690


## BERT (with LoRA)

In [4]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

training_args = TrainingArguments(
    output_dir="distilbert",
    learning_rate=1e-6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16 = True)


In [9]:
model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-multilingual-cased", num_labels=2, id2label=id2label, label2id=label2id
).to(device)

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")

def preprocess_function(examples, batched = True):
    return tokenizer(examples["text"], truncation=True)


dataset = Dataset.from_pandas(df)
train_testvalid = dataset.train_test_split(0.2, seed=42)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(0.5)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

dataset = train_test_valid_dataset
tokenized_text = dataset.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    output_dir="bert",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16 = True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_text["train"],
    eval_dataset=tokenized_text["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/20064 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

/users/eleves-a/2020/meilame.tayebjee/miniconda3/lib/python3.12/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.226400,0.183298,0.929426,0.854560,0.781955,0.942029
2,0.190900,0.198609,0.924242,0.848967,0.756374,0.967391
3,0.172700,0.183502,0.931021,0.858313,0.783259,0.949275
4,0.150700,0.191341,0.926635,0.843003,0.796774,0.894928
5,0.129800,0.180579,0.933014,0.860465,0.794479,0.938406
6,0.127700,0.193539,0.931021,0.847845,0.823932,0.873188
